# Лабораторная работа №4

## Задание 1 . 
###### Провести классификацию найденного датасета, методами наивного Байеса  . В формате Markdown написать пояснения. Объяснить почему были выбраны именно такие гиперпараметры, была ли перекрестная проверка, и т.д.

датасет: [Homicide Reports, 1980-2014](https://www.kaggle.com/datasets/murderaccountability/homicide-reports)

### Загрузка Библиотек

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

### Загрузка данных и предварительная обработка
###### Загрузка данных

In [2]:
N      = 200000 
N_test = 0.99992 

In [3]:
data = pd.read_csv('database.csv',low_memory=False)
subset_data = data.sample(n=N, random_state=42)
# Исключение ненужных столбцов
columns_to_drop = ['Record ID', 'Agency Code', 'Agency Name', 'Agency Type', 'City', 'State', 'Year', 'Month', 'Record Source']
subset_data = subset_data.drop(columns=columns_to_drop)

subset_data.dropna(inplace=True) # Удаление записей с пропущенными значениями
subset_data.head()

,Incident,Crime Type,Crime Solved,Victim Sex,Victim Age,Victim Race,Victim Ethnicity,Perpetrator Sex,Perpetrator Age,Perpetrator Race,Perpetrator Ethnicity,Relationship,Weapon,Victim Count,Perpetrator Count
484984,1,Murder or Manslaughter,No,Male,18,White,Hispanic,Unknown,0,Unknown,Unknown,Unknown,Handgun,1,0
608445,3,Murder or Manslaughter,No,Male,27,White,Not Hispanic,Unknown,0,Unknown,Unknown,Unknown,Firearm,0,0
417612,1,Murder or Manslaughter,Yes,Male,42,Black,Not Hispanic,Male,18,Black,Not Hispanic,Acquaintance,Handgun,0,0
372692,6,Murder or Manslaughter,No,Male,44,White,Not Hispanic,Unknown,0,Unknown,Unknown,Unknown,Handgun,0,0
402028,1,Murder or Manslaughter,Yes,Female,15,White,Unknown,Male,20,White,Unknown,Girlfriend,Rifle,0,0


###### Выбор признаков и целевой переменной

In [4]:
# Выбор признаков и целевой переменной
X = subset_data.drop(columns=['Crime Solved'])
#y = subset_data['Crime Solved']
y = subset_data['Crime Solved'].apply(lambda x: 1 if x == 'Yes' else 0)  # Преобразование целевой переменной в бинарный формат
X = pd.get_dummies(X)

# Масштабирование данных
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

### Разделение данных на тренировочную и тестовую выборки

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=N_test, random_state=42)

### Построение и оценка модели

In [6]:
%%time
param_grid = {'var_smoothing': [1e-9, 1e-8, 1e-7, 1e-6, 1e-5, 1e-4, 1e-3]}
grid_search = GridSearchCV(
    GaussianNB(), 
    param_grid, 
    cv=5,
    scoring='accuracy',
    n_jobs=16
)
grid_search.fit(X_train, y_train)

CPU times: total: 156 ms
Wall time: 10.7 s


GridSearchCV(cv=5, estimator=GaussianNB(), n_jobs=16,
             param_grid={'var_smoothing': [1e-09, 1e-08, 1e-07, 1e-06, 1e-05,
                                           0.0001, 0.001]},
             scoring='accuracy')

###### Лучшая модель

In [7]:
best_params = grid_search.best_params_
print("Best params:", best_params)

Best params: {'var_smoothing': 1e-09}


###### Обучение модели с лучшими гиперпараметрами

In [8]:
%%time
best_var_smoothing = best_params['var_smoothing']
model = GaussianNB(var_smoothing=best_var_smoothing)
model.fit(X_train, y_train)

CPU times: total: 15.6 ms
Wall time: 17.3 ms


GaussianNB()

###### Оценка модели на тестовом наборе

In [9]:
%%time
print("Наилучшие гиперпараметры:")
print(grid_search.best_params_)

y_pred = model.predict(X_test)

print("Матрица ошибок:\n", confusion_matrix(y_test, y_pred))
print("Точность:", accuracy_score(y_test, y_pred))
print("Отчет о классификации:")
print(classification_report(y_test, y_pred))

Наилучшие гиперпараметры:
{'var_smoothing': 1e-09}
Матрица ошибок:
 [[ 56436   3227]
 [  2330 137991]]
Точность: 0.9722127770221618
Отчет о классификации:
              precision    recall  f1-score   support

           0       0.96      0.95      0.95     59663
           1       0.98      0.98      0.98    140321

    accuracy                           0.97    199984
   macro avg       0.97      0.96      0.97    199984
weighted avg       0.97      0.97      0.97    199984

CPU times: total: 1.16 s
Wall time: 1.22 s
